We have the wave equation:
$$
\frac{\partial^{2} u}{\partial t^{2}} = c^{2}\nabla^{2} u
$$ 
defined over a domain $\Omega$. Let the boundary, $\partial \Omega$, but the union of the Neumann ($\Gamma_{N}$) and Dirichlet ($\Gamma_{D}$) boundaries.

Let boudnary conditions be:
$$
\begin{cases}
\nabla u \cdot n = 0 &\text{on }\Gamma_{N}\\
u = \cos(10\pi t)/10\pi &\text{on }\Gamma_{D}
\end{cases}
$$

The infinite dimension weak form is
$$
\int_{\Omega} w \frac{\partial^{2} u}{\partial t^{2}} \,dx = \int_{\Omega}c^{2} \nabla u \cdot \nabla w\,dx
$$
where $w$ is the weighting function in a suitable function space $V$.

We can turn this problem into a first order problem first. Let
$$
v = -\frac{\partial u}{\partial t}
$$
then,
$$
\begin{align}
&\frac{\partial u}{\partial t} = -v\\
&\frac{\partial v}{\partial t} + \nabla^{2} u = 0\\
&p = \sin(10\pi t) &&\text{on }\Gamma_{D}
\end{align}
$$
The weak form is
$$
\int_{\Omega} w \frac{\partial v}{\partial t} \,dx = \int_{\Omega} c^{2} \nabla u \cdot \nabla w\,dx
$$

$\Gamma_{D}$ is edge of the source. 

$\Gamma_{N}$ is everywhere else.

In [ ]:
from dolfin import *
from mshr import *

In [ ]:
plane_w = 2.0/3
plane_h = 8.0/3

wall_w = 0.005/3
wall_h = 0.05/3
slit_h = 0.5/3

plane = Rectangle(Point(plane_w*4, plane_h), Point(-plane_w, -plane_h))
wall1 = Rectangle(Point(wall_w, slit_h), Point(-wall_w, -slit_h))
wall2 = Rectangle(Point(wall_w, plane_h), Point(-wall_w, slit_h+wall_h))
wall3 = Rectangle(Point(wall_w, -plane_h), Point(-wall_w, -slit_h-wall_h))
wall4 = Rectangle(Point(-plane_w, plane_h), Point(-wall_w, slit_h*2))
wall5 = Rectangle(Point(-plane_w, -plane_h), Point(-wall_w, -slit_h*2))

domain = plane - wall1 - wall2 - wall3 - wall4 - wall5
mesh=generate_mesh(domain, 128)

In [ ]:
V = FunctionSpace(mesh, "CG", 2)

p = Function(V, name="p")
phi = Function(V, name="phi")

u = TrialFunction(V)
v = TestFunction(V)

# write the initial conditions:
vtkFile = File("solution/wave.pvd")
# this is zero everywhere
vtkFile << phi

In [ ]:
def source_boundary(x, on_boundary): 
    return on_boundary and near(x[0], -plane_w, DOLFIN_EPS)

bcval = Constant(0.0)
bc = DirichletBC(V, bcval, source_boundary)

In [ ]:
t = 0
dt = 0.001
T = 10.0
step = 0
while t <= T:
    step += 1
    bcval.assign(sin(10*pi*t))

    phi.assign(phi - dt / 2.0 * p)
    solve(u*v*dx == v*p*dx + dt*inner(grad(v), grad(phi))*dx,
          p,
          bc)
    phi.assign(phi - dt / 2.0 * p)

    t += dt
    if step % 10 == 0:
        vtkFile << (phi, t)